# Debug inconsistent schema
2023.02.07

In [14]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.types import StructType, StructField, StringType, LongType, DateType, DoubleType
from pyspark.sql.functions import col, lit, array, explode, arrays_zip, posexplode, \
    sum as sum_, max as max_, min as min_, year, month

In [15]:
# instantiate Spark session
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('process-weather-stations') \
    .getOrCreate()

23/02/09 19:30:32 WARN Utils: Your hostname, hp-ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.1.167 instead (on interface wlp1s0)
23/02/09 19:30:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/09 19:30:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
station_schema = StructType([StructField('STATION', StringType(), True),
 StructField('NAME', StringType(), True),
 StructField('LATITUDE', DoubleType(), True),
 StructField('LONGITUDE', DoubleType(), True),
 StructField('ELEVATION', DoubleType(), True),
 StructField('DATE', DateType(), True),
 StructField('HR00Val', LongType(), True),
 StructField('HR00MF', StringType(), True),
 StructField('HR00QF', StringType(), True),
 StructField('HR00S1', StringType(), True),
 StructField('HR00S2', StringType(), True),
 StructField('HR01Val', LongType(), True),
 StructField('HR01MF', StringType(), True),
 StructField('HR01QF', StringType(), True),
 StructField('HR01S1', StringType(), True),
 StructField('HR01S2', StringType(), True),
 StructField('HR02Val', LongType(), True),
 StructField('HR02MF', StringType(), True),
 StructField('HR02QF', StringType(), True),
 StructField('HR02S1', StringType(), True),
 StructField('HR02S2', StringType(), True),
 StructField('HR03Val', LongType(), True),
 StructField('HR03MF', StringType(), True),
 StructField('HR03QF', StringType(), True),
 StructField('HR03S1', StringType(), True),
 StructField('HR03S2', StringType(), True),
 StructField('HR04Val', LongType(), True),
 StructField('HR04MF', StringType(), True),
 StructField('HR04QF', StringType(), True),
 StructField('HR04S1', StringType(), True),
 StructField('HR04S2', StringType(), True),
 StructField('HR05Val', LongType(), True),
 StructField('HR05MF', StringType(), True),
 StructField('HR05QF', StringType(), True),
 StructField('HR05S1', StringType(), True),
 StructField('HR05S2', StringType(), True),
 StructField('HR06Val', LongType(), True),
 StructField('HR06MF', StringType(), True),
 StructField('HR06QF', StringType(), True),
 StructField('HR06S1', StringType(), True),
 StructField('HR06S2', StringType(), True),
 StructField('HR07Val', LongType(), True),
 StructField('HR07MF', StringType(), True),
 StructField('HR07QF', StringType(), True),
 StructField('HR07S1', StringType(), True),
 StructField('HR07S2', StringType(), True),
 StructField('HR08Val', LongType(), True),
 StructField('HR08MF', StringType(), True),
 StructField('HR08QF', StringType(), True),
 StructField('HR08S1', StringType(), True),
 StructField('HR08S2', StringType(), True),
 StructField('HR09Val', LongType(), True),
 StructField('HR09MF', StringType(), True),
 StructField('HR09QF', StringType(), True),
 StructField('HR09S1', StringType(), True),
 StructField('HR09S2', StringType(), True),
 StructField('HR10Val', LongType(), True),
 StructField('HR10MF', StringType(), True),
 StructField('HR10QF', StringType(), True),
 StructField('HR10S1', StringType(), True),
 StructField('HR10S2', StringType(), True),
 StructField('HR11Val', LongType(), True),
 StructField('HR11MF', StringType(), True),
 StructField('HR11QF', StringType(), True),
 StructField('HR11S1', StringType(), True),
 StructField('HR11S2', StringType(), True),
 StructField('HR12Val', LongType(), True),
 StructField('HR12MF', StringType(), True),
 StructField('HR12QF', StringType(), True),
 StructField('HR12S1', StringType(), True),
 StructField('HR12S2', StringType(), True),
 StructField('HR13Val', LongType(), True),
 StructField('HR13MF', StringType(), True),
 StructField('HR13QF', StringType(), True),
 StructField('HR13S1', StringType(), True),
 StructField('HR13S2', StringType(), True),
 StructField('HR14Val', LongType(), True),
 StructField('HR14MF', StringType(), True),
 StructField('HR14QF', StringType(), True),
 StructField('HR14S1', StringType(), True),
 StructField('HR14S2', StringType(), True),
 StructField('HR15Val', LongType(), True),
 StructField('HR15MF', StringType(), True),
 StructField('HR15QF', StringType(), True),
 StructField('HR15S1', StringType(), True),
 StructField('HR15S2', StringType(), True),
 StructField('HR16Val', LongType(), True),
 StructField('HR16MF', StringType(), True),
 StructField('HR16QF', StringType(), True),
 StructField('HR16S1', StringType(), True),
 StructField('HR16S2', StringType(), True),
 StructField('HR17Val', LongType(), True),
 StructField('HR17MF', StringType(), True),
 StructField('HR17QF', StringType(), True),
 StructField('HR17S1', StringType(), True),
 StructField('HR17S2', StringType(), True),
 StructField('HR18Val', LongType(), True),
 StructField('HR18MF', StringType(), True),
 StructField('HR18QF', StringType(), True),
 StructField('HR18S1', StringType(), True),
 StructField('HR18S2', StringType(), True),
 StructField('HR19Val', LongType(), True),
 StructField('HR19MF', StringType(), True),
 StructField('HR19QF', StringType(), True),
 StructField('HR19S1', StringType(), True),
 StructField('HR19S2', StringType(), True),
 StructField('HR20Val', LongType(), True),
 StructField('HR20MF', StringType(), True),
 StructField('HR20QF', StringType(), True),
 StructField('HR20S1', StringType(), True),
 StructField('HR20S2', StringType(), True),
 StructField('HR21Val', LongType(), True),
 StructField('HR21MF', StringType(), True),
 StructField('HR21QF', StringType(), True),
 StructField('HR21S1', StringType(), True),
 StructField('HR21S2', StringType(), True),
 StructField('HR22Val', LongType(), True),
 StructField('HR22MF', StringType(), True),
 StructField('HR22QF', StringType(), True),
 StructField('HR22S1', StringType(), True),
 StructField('HR22S2', StringType(), True),
 StructField('HR23Val', LongType(), True),
 StructField('HR23MF', StringType(), True),
 StructField('HR23QF', StringType(), True),
 StructField('HR23S1', StringType(), True),
 StructField('HR23S2', StringType(), True),
 StructField('DlySum', LongType(), True),
 StructField('DlySumMF', StringType(), True),
 StructField('DlySumQF', StringType(), True),
 StructField('DlySumS1', StringType(), True),
 StructField('DlySumS2', StringType(), True)])

In [9]:
!ls ../data/raw

csv  parquet


In [17]:
df_t = spark.read.schema(station_schema).parquet('../data/raw/parquet/USC00465341.parquet')

In [11]:
df_t.printSchema()

root
 |-- STATION: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- ELEVATION: double (nullable = true)
 |-- DATE: date (nullable = true)
 |-- HR00Val: long (nullable = true)
 |-- HR00MF: string (nullable = true)
 |-- HR00QF: string (nullable = true)
 |-- HR00S1: string (nullable = true)
 |-- HR00S2: string (nullable = true)
 |-- HR01Val: long (nullable = true)
 |-- HR01MF: string (nullable = true)
 |-- HR01QF: string (nullable = true)
 |-- HR01S1: string (nullable = true)
 |-- HR01S2: string (nullable = true)
 |-- HR02Val: long (nullable = true)
 |-- HR02MF: string (nullable = true)
 |-- HR02QF: string (nullable = true)
 |-- HR02S1: string (nullable = true)
 |-- HR02S2: string (nullable = true)
 |-- HR03Val: long (nullable = true)
 |-- HR03MF: string (nullable = true)
 |-- HR03QF: string (nullable = true)
 |-- HR03S1: string (nullable = true)
 |-- HR03S2: string (nullable = true)
 |-- HR0

In [19]:
df_t.write.mode('overwrite').parquet('test_out')

In [8]:
df = spark.read.schema(station_schema).parquet('s3a://weather-data-kpde/raw/*')

23/02/02 19:49:50 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
23/02/02 19:49:50 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.
23/02/02 19:49:50 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.
23/02/02 19:49:51 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.


In [6]:
import pyarrow as pa
import pyarrow.csv as pv
import pyarrow.parquet as pq

In [14]:
dir(pa)

['Array',
 'ArrowCancelled',
 'ArrowCapacityError',
 'ArrowException',
 'ArrowIOError',
 'ArrowIndexError',
 'ArrowInvalid',
 'ArrowKeyError',
 'ArrowMemoryError',
 'ArrowNotImplementedError',
 'ArrowSerializationError',
 'ArrowTypeError',
 'BaseExtensionType',
 'BinaryArray',
 'BinaryScalar',
 'BooleanArray',
 'BooleanScalar',
 'Buffer',
 'BufferOutputStream',
 'BufferReader',
 'BufferedInputStream',
 'BufferedOutputStream',
 'BuildInfo',
 'ChunkedArray',
 'Codec',
 'CompressedInputStream',
 'CompressedOutputStream',
 'DataType',
 'Date32Array',
 'Date32Scalar',
 'Date64Array',
 'Date64Scalar',
 'Decimal128Array',
 'Decimal128Scalar',
 'Decimal128Type',
 'Decimal256Array',
 'Decimal256Scalar',
 'Decimal256Type',
 'DenseUnionType',
 'DeserializationCallbackError',
 'DictionaryArray',
 'DictionaryMemo',
 'DictionaryScalar',
 'DictionaryType',
 'DoubleScalar',
 'DurationArray',
 'DurationScalar',
 'DurationType',
 'ExtensionArray',
 'ExtensionScalar',
 'ExtensionType',
 'Field',
 'FixedS

In [2]:
# rewrite csv as proper parquet
parquet_schema = pa.schema([
 ('STATION', pa.string()),
 ('NAME', pa.string()),
 ('LATITUDE', pa.float64()),
 ('LONGITUDE', pa.float64()),
 ('ELEVATION', pa.float64()),
 ('DATE', pa.date64()),
 ('HR00Val', pa.int64()),
 ('HR00MF', pa.string()),
 ('HR00QF', pa.string()),
 ('HR00S1', pa.string()),
 ('HR00S2', pa.string()),
 ('HR01Val', pa.int64()),
 ('HR01MF', pa.string()),
 ('HR01QF', pa.string()),
 ('HR01S1', pa.string()),
 ('HR01S2', pa.string()),
 ('HR02Val', pa.int64()),
 ('HR02MF', pa.string()),
 ('HR02QF', pa.string()),
 ('HR02S1', pa.string()),
 ('HR02S2', pa.string()),
 ('HR03Val', pa.int64()),
 ('HR03MF', pa.string()),
 ('HR03QF', pa.string()),
 ('HR03S1', pa.string()),
 ('HR03S2', pa.string()),
 ('HR04Val', pa.int64()),
 ('HR04MF', pa.string()),
 ('HR04QF', pa.string()),
 ('HR04S1', pa.string()),
 ('HR04S2', pa.string()),
 ('HR05Val', pa.int64()),
 ('HR05MF', pa.string()),
 ('HR05QF', pa.string()),
 ('HR05S1', pa.string()),
 ('HR05S2', pa.string()),
 ('HR06Val', pa.int64()),
 ('HR06MF', pa.string()),
 ('HR06QF', pa.string()),
 ('HR06S1', pa.string()),
 ('HR06S2', pa.string()),
 ('HR07Val', pa.int64()),
 ('HR07MF', pa.string()),
 ('HR07QF', pa.string()),
 ('HR07S1', pa.string()),
 ('HR07S2', pa.string()),
 ('HR08Val', pa.int64()),
 ('HR08MF', pa.string()),
 ('HR08QF', pa.string()),
 ('HR08S1', pa.string()),
 ('HR08S2', pa.string()),
 ('HR09Val', pa.int64()),
 ('HR09MF', pa.string()),
 ('HR09QF', pa.string()),
 ('HR09S1', pa.string()),
 ('HR09S2', pa.string()),
 ('HR10Val', pa.int64()),
 ('HR10MF', pa.string()),
 ('HR10QF', pa.string()),
 ('HR10S1', pa.string()),
 ('HR10S2', pa.string()),
 ('HR11Val', pa.int64()),
 ('HR11MF', pa.string()),
 ('HR11QF', pa.string()),
 ('HR11S1', pa.string()),
 ('HR11S2', pa.string()),
 ('HR12Val', pa.int64()),
 ('HR12MF', pa.string()),
 ('HR12QF', pa.string()),
 ('HR12S1', pa.string()),
 ('HR12S2', pa.string()),
 ('HR13Val', pa.int64()),
 ('HR13MF', pa.string()),
 ('HR13QF', pa.string()),
 ('HR13S1', pa.string()),
 ('HR13S2', pa.string()),
 ('HR14Val', pa.int64()),
 ('HR14MF', pa.string()),
 ('HR14QF', pa.string()),
 ('HR14S1', pa.string()),
 ('HR14S2', pa.string()),
 ('HR15Val', pa.int64()),
 ('HR15MF', pa.string()),
 ('HR15QF', pa.string()),
 ('HR15S1', pa.string()),
 ('HR15S2', pa.string()),
 ('HR16Val', pa.int64()),
 ('HR16MF', pa.string()),
 ('HR16QF', pa.string()),
 ('HR16S1', pa.string()),
 ('HR16S2', pa.string()),
 ('HR17Val', pa.int64()),
 ('HR17MF', pa.string()),
 ('HR17QF', pa.string()),
 ('HR17S1', pa.string()),
 ('HR17S2', pa.string()),
 ('HR18Val', pa.int64()),
 ('HR18MF', pa.string()),
 ('HR18QF', pa.string()),
 ('HR18S1', pa.string()),
 ('HR18S2', pa.string()),
 ('HR19Val', pa.int64()),
 ('HR19MF', pa.string()),
 ('HR19QF', pa.string()),
 ('HR19S1', pa.string()),
 ('HR19S2', pa.string()),
 ('HR20Val', pa.int64()),
 ('HR20MF', pa.string()),
 ('HR20QF', pa.string()),
 ('HR20S1', pa.string()),
 ('HR20S2', pa.string()),
 ('HR21Val', pa.int64()),
 ('HR21MF', pa.string()),
 ('HR21QF', pa.string()),
 ('HR21S1', pa.string()),
 ('HR21S2', pa.string()),
 ('HR22Val', pa.int64()),
 ('HR22MF', pa.string()),
 ('HR22QF', pa.string()),
 ('HR22S1', pa.string()),
 ('HR22S2', pa.string()),
 ('HR23Val', pa.int64()),
 ('HR23MF', pa.string()),
 ('HR23QF', pa.string()),
 ('HR23S1', pa.string()),
 ('HR23S2', pa.string()),
 ('DlySum', pa.int64()),
 ('DlySumMF', pa.string()),
 ('DlySumQF', pa.string()),
 ('DlySumS1', pa.string()),
 ('DlySumS2', pa.string())
])

In [11]:
table = pv.read_csv('../data/raw/csv/USC00465341.csv')
# pq.write_table(table, dir_data / 'parquet' / file_parquet)

In [12]:
table = table.cast(parquet_schema)

In [13]:
pq.write_table(table, '../data/raw/parquet/USC00465341.parquet')

In [10]:
!ls ../data/raw/csv